In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from uszipcode import SearchEngine
from uszipcode import Zipcode


# Extract

In [3]:
filesDir="../Resources/"
raw_data_df = pd.read_csv(f"{filesDir}sanFranCrime.csv", quotechar = '"')

In [4]:
# Separate data into crime, geographic and time
crime_df = raw_data_df[['PdId','IncidntNum', 'Category', 'Descript','Resolution']]
crime_df.head()

,PdId,IncidntNum,Category,Descript,Resolution
0,12005827212120,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,"ARREST, BOOKED"
1,12005827212168,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE","ARREST, BOOKED"
2,14105926363010,141059263,WARRANTS,WARRANT ARREST,"ARREST, BOOKED"
3,16001366271000,160013662,NON-CRIMINAL,LOST PROPERTY,NONE
4,16000274071000,160002740,NON-CRIMINAL,LOST PROPERTY,NONE


In [5]:
geographic = raw_data_df[['PdId','PdDistrict','Address', 'X', 'Y', 'Location']]
geographic.head()

,PdId,PdDistrict,Address,X,Y,Location
0,12005827212120,SOUTHERN,800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
1,12005827212168,SOUTHERN,800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
2,14105926363010,BAYVIEW,KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)"
3,16001366271000,TENDERLOIN,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)"
4,16000274071000,MISSION,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)"


In [6]:
crime_dates = raw_data_df[['PdId','DayOfWeek', 'Date', 'Time']]
crime_dates.head()

,PdId,DayOfWeek,Date,Time
0,12005827212120,Friday,01/29/2016 12:00:00 AM,11:00
1,12005827212168,Friday,01/29/2016 12:00:00 AM,11:00
2,14105926363010,Monday,04/25/2016 12:00:00 AM,14:59
3,16001366271000,Tuesday,01/05/2016 12:00:00 AM,23:50
4,16000274071000,Friday,01/01/2016 12:00:00 AM,00:30


# Transform

In [7]:
# Convert Location using X and Y coordinates to ZipCodes in order to perform analysis.
# https://uszipcode.readthedocs.io/index.html#example-usage
# This took about 30 min to run FYI

search = SearchEngine(simple_zipcode=True)

def convertToZip(df):
    '''Creates a zipcode column from Y and X coordinates in a dataframe. uszipcode returns multiple datapoints by default so only selecting the zipcode         in the return value'''

    result = search.by_coordinates(df['Y'], df['X'])
    return result[0].zipcode

geographic['ZipCode'] = geographic.apply(convertToZip, axis=1)
geographic.head()

,PdId,PdDistrict,Address,X,Y,Location,ZipCode
0,12005827212120,SOUTHERN,800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",94103
1,12005827212168,SOUTHERN,800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",94103
2,14105926363010,BAYVIEW,KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",94124
3,16001366271000,TENDERLOIN,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",94108
4,16000274071000,MISSION,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",94102


# Generate csv files to be loaded in DB

In [12]:
geographic.to_csv("../Resources/geographic.csv",index=False)

In [13]:
crime_dates.to_csv("../Resources/crime_dates.csv",index=False)

In [14]:
crime_df.to_csv("../Resources/crime_df.csv",index=False)